
### 1、用fasterrcnn的swin模型进行了训练。
### 2、数据全部放train里面了
### 3、最后提交的时候要进行排序，不然分数会很低。
### myconfig为配置文件。own_model为模型
代码如下:


In [1]:
# 解压PaddleDetection
# !unzip work/PaddleDetection.zip -d work

# 下载PaddleDetection
%cd work

!git clone https://gitee.com/paddlepaddle/PaddleDetection.git -b release/2.3 


/home/aistudio/work
fatal: 目标路径 'PaddleDetection' 已经存在，并且不是一个空目录。


In [2]:
# 进入PaddleDetection
%cd /home/aistudio/work/PaddleDetection


/home/aistudio/work/PaddleDetection


In [3]:
# 升级pip
!pip install --upgrade pip -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 1.7MB 7.9MB/s eta 0:00:01
  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:
      Successfully uninstalled pip-19.2.3


In [4]:
# 安装其它依赖
!pip install -r /home/aistudio/work/PaddleDetection/requirements.txt  


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.1 MB 5.0 MB/s            
     |████████████████████████████████| 106 kB 453 kB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 1.5 MB 6.0 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 151 kB 8.3 MB/s            
     |████████████████████████████████| 41 kB 982 kB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 280 kB 17.7 MB/s            
     |████████████████████████████████| 50 kB 20.9 MB/s            
     |████████████████████████████████| 98 kB 416 kB/s             
     |████████████████████████████████| 99 kB 3.3 MB/s             
  Preparing metadata (setup.py) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0.3-cp37-cp37m-linux_x86_64.whl size=273732 sha256=1dba63ae1dd172579985d86ceb6eb7ca178bb713615d7b8c3c5db8f

In [5]:
# 临时环境安装
!pip install pycocotools -i https://mirror.baidu.com/pypi/simple
!pip install lap -i https://mirror.baidu.com/pypi/simple


Looking in indexes: https://mirror.baidu.com/pypi/simple
Looking in indexes: https://mirror.baidu.com/pypi/simple


### 2. 处理数据集

In [20]:
# 解压文件并移除多余的目录
! unzip /home/aistudio/data/data105746/train.zip -d /home/aistudio/data/steel
#! rm -f -r /home/aistudio/work/steel __MACOSX
! unzip /home/aistudio/data/data105747/test.zip -d /home/aistudio/data/steel
#! rm -f -r /home/aistudio/work/steel __MACOSX

Archive:  /home/aistudio/data/data105746/train.zip
   creating: /home/aistudio/data/steel/train/
  inflating: /home/aistudio/data/steel/train/.DS_Store  
   creating: /home/aistudio/data/steel/__MACOSX/
   creating: /home/aistudio/data/steel/__MACOSX/train/
  inflating: /home/aistudio/data/steel/__MACOSX/train/._.DS_Store  
   creating: /home/aistudio/data/steel/train/IMAGES/
  inflating: /home/aistudio/data/steel/train/IMAGES/63.jpg  
   creating: /home/aistudio/data/steel/__MACOSX/train/IMAGES/
  inflating: /home/aistudio/data/steel/__MACOSX/train/IMAGES/._63.jpg  
  inflating: /home/aistudio/data/steel/train/IMAGES/823.jpg  
  inflating: /home/aistudio/data/steel/__MACOSX/train/IMAGES/._823.jpg  
  inflating: /home/aistudio/data/steel/train/IMAGES/189.jpg  
  inflating: /home/aistudio/data/steel/__MACOSX/train/IMAGES/._189.jpg  
  inflating: /home/aistudio/data/steel/train/IMAGES/77.jpg  
  inflating: /home/aistudio/data/steel/__MACOSX/train/IMAGES/._77.jpg  
  inflating: /home/aist

In [8]:
# 安装paddlex 用于数据格式转换
#!pip install "numpy>1.19.5" -i https://mirror.baidu.com/pypi/simple 
!pip install "paddlex>2.0.0" -i https://mirror.baidu.com/pypi/simple 

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 1.6 MB 7.3 MB/s            
     |████████████████████████████████| 2.7 MB 26.6 MB/s            
     |████████████████████████████████| 6.8 MB 14.7 MB/s            
     |████████████████████████████████| 310 kB 19.5 MB/s            
  Attempting uninstall: visualdl
    Found existing installation: visualdl 2.2.0
    Uninstalling visualdl-2.2.0:
      Successfully uninstalled visualdl-2.2.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blackhole 1.0.1 requires numpy<=1.19.5, but you have numpy 1.20.3 which is incompatible.


In [21]:
# 修改见文件名字 JPEGImages  Annotations

!mv /home/aistudio/data/steel/train/ANNOTATIONS  /home/aistudio/data/steel/train/Annotations
!mv /home/aistudio/data/steel/train/IMAGES  /home/aistudio/data/steel/train/JPEGImages


#使用paddleX拆分数据集
!paddlex --split_dataset --format VOC --dataset_dir /home/aistudio/data/steel/train --val_value 0.001 --test_value 0.0

In [23]:
%cd /home/aistudio/work/PaddleDetection/

/home/aistudio/work/PaddleDetection


In [24]:
#转换train
!python tools/x2coco.py \
        --dataset_type voc \
        --voc_anno_dir /home/aistudio/data/steel/train/ \
--voc_anno_list /home/aistudio/data/steel/train/train_list.txt \
--voc_label_list /home/aistudio/data/steel/train/labels.txt \
--voc_out_name /home/aistudio/data/steel/train/voc_train.json

Start converting !
100%|████████████████████████████████████| 1399/1399 [00:00<00:00, 16408.05it/s]


In [25]:
#转换test
!python tools/x2coco.py \
        --dataset_type voc \
        --voc_anno_dir /home/aistudio/data/steel/train/ \
--voc_anno_list /home/aistudio/data/steel/train/val_list.txt \
--voc_label_list /home/aistudio/data/steel/train/labels.txt \
--voc_out_name /home/aistudio/data/steel/train/voc_val.json

Start converting !
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 3908.95it/s]


In [26]:
!rm -r /home/aistudio/data/steel/train/Annotations/*
!mv /home/aistudio/data/steel/train/*.json /home/aistudio/data/steel/train/Annotations/

### 3. 配置好训练文件

### 4. 训练及评估

In [34]:

!python tools/train.py -c configs/faster_rcnn/faster_rcnn_swin_tiny_fpn_3x_coco.yml --use_vdl=true --vdl_log_dir=vdl_dir/scalar --eval

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:130: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
W1229 17:53:53.072952 22135 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W1229 17:53:53.077612 22135 device_context.cc:465] device: 0, cuDNN Version: 7.6.
[12/29 17:53:57] ppdet.engine INFO: Epoch: [0] [  0/174] learning_rate: 0.000010 loss_rpn_cls: 0.699856 loss_rpn_reg: 0.244360 loss_bbox_cls: 1.869460 loss_bbox_reg: 0.003470 loss: 2.817145 eta: 2:49:17 batch_cost: 1.3900 data_cost: 0.0004 ips: 5

In [ ]:
# cascade_mask_rcnn_r50_vd_fpn_ssld_2x_coco.yml
# ppyolov2_r101vd_dcn_365e_coco.yml

In [ ]:
# 单卡断点续训
#!python tools/train.py -c configs/yolov3/yolov3_darknet53_270e_voc.yml \
#                       -r /home/aistudio/work/output/yolov3_darknet53_270e_voc/1134 \
#                       --use_vdl=true \
#                      --vdl_log_dir=vdl_dir/scalar \
#                       --eval

# !python tools/train.py -c configs/ppyolo/ppyolov2_r50vd_dcn_voc.yml \
#                        -r /home/aistudio/work/output/ppyolov2_r50vd_dcn_voc/1204 \
#                        --eval  
#                        #--use_vdl=true \
#                        #--vdl_log_dir=vdl_dir/scalar \
#                        #--eval                     

### 5. 生成比赛数据

In [39]:
# 推理图像和生成txt文件
# !python tools/infer.py -c configs/yolov3/yolov3_darknet53_270e_voc.yml -o weights=/home/aistudio/work/output/yolov3_darknet53_270e_voc/best_model --infer_dir=/home/aistudio/data/steel/test/IMAGES/ --output_dir=/home/aistudio/data/steel/infer_output --draw_threshold=0.0001 --save_txt=True

!python tools/infer.py -c  configs/faster_rcnn/faster_rcnn_swin_tiny_fpn_3x_coco.yml \
-o weights=/home/aistudio/work/PaddleDetection/output/faster_rcnn_swin_tiny_fpn_3x_coco/34 \
--infer_dir=/home/aistudio/data/steel/test/IMAGES/ \
--output_dir=/home/aistudio/data/steel/infer_output\
--draw_threshold=0.005 --save_txt=True

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:130: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
W1229 20:25:35.989657 31762 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W1229 20:25:35.994923 31762 device_context.cc:465] device: 0, cuDNN Version: 7.6.
[12/29 20:25:39] ppdet.utils.checkpoint INFO: Finish loading model weights: /home/aistudio/work/PaddleDetection/output/faster_rcnn_swin_tiny_fpn_3x_coco/34.pdparams
[12/29 20:25:39] train INFO: Found 400 inference images in total.
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
[12/29 20:26:08

In [40]:
import csv
import os
headers = ['image_id','bbox','category_id','confidence']
classList = ['crazing','inclusion','pitted_surface','scratches','patches','rolled-in_scale']
rows = []

rootdir = '/home/aistudio/data/steel/infer_output'
list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
for i in range(0,len(list)):
       path = os.path.join(rootdir,list[i])
       if os.path.isfile(path) and path.endswith('txt'):
           txtFile = open(path)
           print(path)
           result = txtFile.readlines()
           for r in result:
               ls = r.split(' ')
               Cls = ls[0]
               sco = float(ls[1])
               xmin = float(ls[2])
               ymin = float(ls[3])
               w = float(ls[4])
               h = float(ls[5])
               xmax = xmin+w
               ymax = ymin+h
               clsID = classList.index(Cls)
               imgID = list[i][:-4]
               row = [imgID,[xmin,ymin,xmax,ymax],clsID,sco]
               rows.append(row)
with open('submission.csv','w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)

/home/aistudio/data/steel/infer_output/1445.txt
/home/aistudio/data/steel/infer_output/1711.txt
/home/aistudio/data/steel/infer_output/1733.txt
/home/aistudio/data/steel/infer_output/1652.txt
/home/aistudio/data/steel/infer_output/1764.txt
/home/aistudio/data/steel/infer_output/1520.txt
/home/aistudio/data/steel/infer_output/1798.txt
/home/aistudio/data/steel/infer_output/1461.txt
/home/aistudio/data/steel/infer_output/1569.txt
/home/aistudio/data/steel/infer_output/1531.txt
/home/aistudio/data/steel/infer_output/1497.txt
/home/aistudio/data/steel/infer_output/1553.txt
/home/aistudio/data/steel/infer_output/1607.txt
/home/aistudio/data/steel/infer_output/1574.txt
/home/aistudio/data/steel/infer_output/1697.txt
/home/aistudio/data/steel/infer_output/1429.txt
/home/aistudio/data/steel/infer_output/1748.txt
/home/aistudio/data/steel/infer_output/1573.txt
/home/aistudio/data/steel/infer_output/1455.txt
/home/aistudio/data/steel/infer_output/1502.txt
/home/aistudio/data/steel/infer_output/1

In [2]:
import pandas as pd
datafile = pd.read_csv('/home/aistudio/work/PaddleDetection/submission.csv')
# 按照列值排序
data = datafile.sort_values(by="image_id", ascending=True)
data.to_csv('submission_final.csv', mode='a+', index=False)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 